In [54]:
import requests, time, json
from datetime import date, datetime
import pandas as pd
import numpy as np

In [38]:
data = pd.concat([pd.read_csv('data/%d.csv' % year, usecols=['Year','Course_#','Permanent_Tournament_#',
                                                             'Round','Hole','Player_#','Shot','Date','Time'])
                  for year in range(2003,2018)])

In [39]:
data.head()

,Year,Player_#,Course_#,Permanent_Tournament_#,Round,Hole,Shot,Time,Date
0,2003,1014,101,1,1,1,1,752,02/27/2003
1,2003,1014,101,1,1,1,2,757,02/27/2003
2,2003,1014,101,1,1,1,3,800,02/27/2003
3,2003,1014,101,1,1,2,1,806,02/27/2003
4,2003,1014,101,1,1,2,2,815,02/27/2003


In [5]:
courses = pd.read_csv('courses.csv')

In [6]:
courses.head()

,Course_#,Course_Name,Latitude,Longitude,TimeZone
0,6,Waialae CC,21.270883,-157.776195,10
1,656,Plantation Course at Kapalua,21.007104,-156.641127,10
2,716,Shaughnessy G&CC,49.233298,-123.210176,7
3,717,Harding Park GC,37.724119,-122.492396,7
4,717,TPC Harding Park,37.724119,-122.492396,7


In [9]:
loc_dict = dict((c,(la,lo)) for c,la,lo in zip(courses['Course_#'],courses['Latitude'],courses['Longitude']))
time_zone_dict = dict((c,tz) for c,tz in zip(courses['Course_#'],courses['TimeZone']))

In [42]:
data['nearest_hour'] = (data['Time']/100.0).apply(round)
data.loc[data.nearest_hour==24,'nearest_hour'] = 23
data.loc[data.nearest_hour==0,'nearest_hour'] = 12

In [43]:
data.nearest_hour.value_counts()

13.0    1993720
12.0    1961312
11.0    1845111
14.0    1790218
10.0    1616130
15.0    1499426
9.0     1298370
16.0    1198808
17.0     767736
8.0      764307
18.0     340394
7.0      193912
19.0     110057
20.0      21970
6.0         502
21.0        364
4.0          64
23.0         32
3.0          23
2.0           9
5.0           9
22.0          9
1.0           3
Name: nearest_hour, dtype: int64

In [25]:
d = data.drop_duplicates(subset=['Course_#','Date']).copy()

In [18]:
d.shape

(2298, 10)

In [35]:
weather_dict = {}
for row_ind,row in d.iterrows():
    la,lo = loc_dict[row['Course_#']]
    month,day,year = map(int,row['Date'].split('/'))
    date_ = date(year,month,day)
    try:
        timestamp = time.mktime(date_.timetuple()) - 4*60*60 ##eastern time
    except:
        print row['Date']
        continue
    request = 'https://api.darksky.net/forecast/412f70f063a8ba73446829ee76c02d9a/%g,%g,%d' % (la,lo,timestamp)
    response = requests.get(request)
    if response.status_code!=200:
        print 'Error', (row['Course_#'],row['Date']), response.status_code
        continue
    weather_dict[(row['Course_#'],row['Date'])] = json.loads(response.content)['hourly']

12/30/1899


In [45]:
d = data.drop_duplicates(subset=['Course_#','Date','nearest_hour']).copy()

In [74]:
keys

[u'apparentTemperature',
 u'cloudCover',
 u'dewPoint',
 u'humidity',
 u'precipIntensity',
 u'precipProbability',
 u'precipType',
 u'pressure',
 u'summary',
 u'temperature',
 u'uvIndex',
 u'visibility',
 u'windBearing',
 u'windSpeed']

In [70]:
hourly_dict = {}
keys = set(key_ for key in weather_dict for dict_ in weather_dict[key]['data'] for key_ in dict_ )
keys = sorted([key for key in keys if key not in ('time','icon')])
for row_ind,row in d.iterrows():
    hour = row['nearest_hour']
    if (row['Course_#'],row['Date']) not in weather_dict:
        print (row['Course_#'],row['Date'])
        continue
    dict_ = weather_dict[(row['Course_#'],row['Date'])]['data'][int(hour)]
    hourly_dict[(row['Course_#'],row['Date'],row['nearest_hour'])] = [dict_[key] if key in dict_ else np.nan
                                                                      for key in keys]

(734, '12/30/1899')


In [82]:
for year in range(2003,2018):
    data = pd.read_csv('data/%d.csv' % year)
    data['nearest_hour'] = (data['Time']/100.0).apply(round)
    data.loc[data.nearest_hour==24,'nearest_hour'] = 23
    data.loc[data.nearest_hour==0,'nearest_hour'] = 12
    for u,key in enumerate(keys):
        data[key] = [hourly_dict[(course,date,hour)][u]
                     if (course,date,hour) in hourly_dict else np.nan
                     for course,date,hour in zip(data['Course_#'],data['Date'],data['nearest_hour'])]
    print year
    print data[keys].isnull().sum()

2003
apparentTemperature         0
cloudCover               2243
dewPoint                    0
humidity                    0
precipIntensity         33840
precipProbability       33840
precipType             868353
pressure                    0
summary                     0
temperature                 0
uvIndex                  2243
visibility                  0
windBearing                67
windSpeed                   0
dtype: int64
2004
apparentTemperature         0
cloudCover               3396
dewPoint                    0
humidity                    0
precipIntensity         52543
precipProbability       52543
precipType             962859
pressure                    0
summary                     0
temperature                 0
uvIndex                  3396
visibility                  0
windBearing              1158
windSpeed                1158
dtype: int64


/Users/adamlevin/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2005
apparentTemperature         0
cloudCover                  0
dewPoint                    0
humidity                    0
precipIntensity         32873
precipProbability       32873
precipType             984057
pressure                    0
summary                     0
temperature                 0
uvIndex                     0
visibility                  0
windBearing                 0
windSpeed                   0
dtype: int64
2006
apparentTemperature         0
cloudCover                820
dewPoint                    0
humidity                    0
precipIntensity         49258
precipProbability       49258
precipType             964371
pressure                    0
summary                     0
temperature                 0
uvIndex                   820
visibility                  0
windBearing                 0
windSpeed                   0
dtype: int64
2007
apparentTemperature         0
cloudCover               2072
dewPoint                    0
humidity                    0

In [29]:
data['year'].value_counts()

2005    1165847
2006    1159804
2004    1144480
2008    1117610
2007    1103201
2003    1091907
2010    1043983
2009    1015724
2014    1013357
2011    1008666
2015    1006351
2012     991313
2016     985162
2013     983842
2017     571238
1899          1
Name: year, dtype: int64

In [8]:
data.groupby('Year').apply(lambda x: (x['Date']=='00/00/0000').mean())

Year
2003    1.000000
2004    1.000000
2005    0.017691
2006    0.000000
2007    0.000000
2008    0.000000
2009    0.000000
2010    0.000000
2011    0.000000
2012    0.000000
2013    0.000000
2014    0.000000
2015    0.000000
2016    0.000000
2017    0.000000
dtype: float64

In [33]:
(data.Time==0).mean()

4.5447208976524956e-07

In [84]:
data.shape

(681645, 78)

In [87]:
pd.options.display.max_columns = 78

In [90]:
data.tail(30)

,Tour_Code,Tour_Description,Year,Tourn.#,Player_#,Course_#,Permanent_Tournament_#,Player_First_Name,Player_Last_Name,Round,Tournament_Name,Course_Name,Hole,Hole_Score,Par_Value,Yardage,Shot,Shot_Type(S/P/D),#_of_Strokes,From_Location(Scorer),From_Location(Enhanced),To_Location(Scorer),To_Location(Enhanced),Distance,Distance_to_Pin,In_the_Hole_Flag,Around_the_Green_Flag,1st_Putt_Flag,Distance_to_Hole_after_the_Shot,Time,Lie,Elevation,Slope,End_X_Coordinate,End_Y_Coordinate,End_Z_Coordinate,Distance_from_Center,Distance_from_Edge,Date,Left/Right,Strokes_Gained/Baseline,Strokes_Gained_Category,Recovery_Shot,Penalty_Shots,start_loc_equal_to_prev_start_loc,Start_X_Coordinate,Start_Y_Coordinate,Start_Z_Coordinate,Real_Shots,last_shot_mask,Stroke,Strokes_from_starting_location,from_the_tee_box_mask,Distance_from_hole,Cat,Green_to_work_with,Cluster,Cluster_Green_X,Cluster_Green_Y,Cluster_Tee_X,Cluster_Tee_Y,loc_string,loc_string_hole,nearest_hour,apparentTemperature,cloudCover,dewPoint,humidity,precipIntensity,precipProbability,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windSpeed
681615,R,PGA TOUR,2017,70,50092,776,493,Zach,Wright,1,The RSM Classic,Sea Island Resort (Seaside),10,4,4,418,3,S,1,Green,Unmapped,Green,Unmapped,268,287,N,N,Y,19,1255,Good,With,Level,9998.802000,8022.040000,16.581,14,208,11/17/2016,R,-0.084,Putting,No,0,False,10021.152000,8021.580000,15.643,3,False,3,2,False,24.027735,Green,NaN,0,10003.560995,8014.643787,10794.561140,7034.956652,776-10-19-1,776-10-2-1,13.0,71.28,0.00,47.27,0.42,0.0,0.0,NaN,1014.73,Clear,71.28,3.0,9.25,142.0,3.82
681616,R,PGA TOUR,2017,70,50092,776,493,Zach,Wright,1,The RSM Classic,Sea Island Resort (Seaside),10,4,4,418,4,S,1,Green,Unmapped,NaN,NaN,19,19,Y,N,N,0,1255,Good,With,Level,9997.128191,8022.014331,16.634,0,0,11/17/2016,NaN,0.005,Putting,No,0,False,9998.802000,8022.040000,16.581,4,True,4,1,False,1.674006,Green,NaN,0,10003.560995,8014.643787,10794.561140,7034.956652,776-10-3-1,776-10-2-1,13.0,71.28,0.00,47.27,0.42,0.0,0.0,NaN,1014.73,Clear,71.28,3.0,9.25,142.0,3.82
681617,R,PGA TOUR,2017,70,50092,776,493,Zach,Wright,1,The RSM Classic,Sea Island Resort (Seaside),11,4,4,425,1,S,1,Tee Box,NaN,Fairway,Left Fairway,11749,15336,N,N,N,3623,1259,Good,With,Level,9473.261000,8463.476000,10.575,332,224,11/17/2016,L,0.263,Off the Tee,No,0,False,10340.934823,8009.801271,NaN,1,False,1,4,True,1279.843793,Tee Box,41.728333,0,9230.420909,8631.398023,10338.602433,8011.312130,776-11-13-8,776-11-2-1,13.0,71.28,0.00,47.27,0.42,0.0,0.0,NaN,1014.73,Clear,71.28,3.0,9.25,142.0,3.82
681618,R,PGA TOUR,2017,70,50092,776,493,Zach,Wright,1,The RSM Classic,Sea Island Resort (Seaside),11,4,4,425,2,S,1,Fairway,Left Fairway,Green,Unmapped,3676,3623,N,N,N,284,1306,Good,With,Level,9232.513000,8652.855000,12.695,285,291,11/17/2016,R,-0.140,Approach the Green,No,0,False,9473.261000,8463.476000,10.575,2,False,2,3,False,302.059463,Fairway,24.976194,0,9230.420909,8631.398023,10338.602433,8011.312130,776-11-12-4,776-11-2-1,13.0,71.28,0.00,47.27,0.42,0.0,0.0,NaN,1014.73,Clear,71.28,3.0,9.25,142.0,3.82
681619,R,PGA TOUR,2017,70,50092,776,493,Zach,Wright,1,The RSM Classic,Sea Island Resort (Seaside),11,4,4,425,3,S,1,Green,Unmapped,Green,Unmapped,333,284,N,N,Y,51,1309,Good,With,Level,9222.486000,8626.980000,12.643,44,172,11/17/2016,L,-0.244,Putting,No,0,False,9232.513000,8652.855000,12.695,3,False,3,2,False,23.831179,Green,NaN,0,9230.420909,8631.398023,10338.602433,8011.312130,776-11-1-1,776-11-2-1,13.0,71.28,0.00,47.27,0.42,0.0,0.0,NaN,1014.73,Clear,71.28,3.0,9.25,142.0,3.82
681620,R,PGA TOUR,2017,70,50092,776,493,Zach,Wright,1,The RSM Classic,Sea Island Resort (Seaside),11,4,4,425,4,S,1,Green,Unmapped,NaN,NaN,51,51,Y,N,N,0,1312,Good,With,Level,9222.170465,8631.385090,12.709,0,0,11/17/2016,NaN,0.162,Putting,No,0,False,9222.486000,8626.980000,12.643,4,True,4,1,False,4.416376,Green,NaN,0,9230.420909,8631.398023,10338.602433,8011.312130,776-11-5-1,776-11-2-1,13.0,71.28,0.00,47.27,0.42,0.0,0.0,NaN,1014.73,C

In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681645 entries, 0 to 681644
Data columns (total 78 columns):
Tour_Code                            681645 non-null object
Tour_Description                     681645 non-null object
Year                                 681645 non-null int64
Tourn.#                              681645 non-null int64
Player_#                             681645 non-null int64
Course_#                             681645 non-null int64
Permanent_Tournament_#               681645 non-null int64
Player_First_Name                    681645 non-null object
Player_Last_Name                     681645 non-null object
Round                                681645 non-null int64
Tournament_Name                      681645 non-null object
Course_Name                          681645 non-null object
Hole                                 681645 non-null int64
Hole_Score                           681645 non-null object
Par_Value                            681645 non-null int